In [10]:
import pandas as pd
import yfinance as yf
import datetime
from IPython.display import display, clear_output
import ipywidgets as widgets

def backtest_breakout_strategy(X, Y, Z, ticker="AAPL", start_date="2020-01-01", end_date="2021-01-01"):
    # Download historical data
    df = yf.download(ticker, start=start_date, end=end_date)

    # Print diagnostic info
    print("Downloaded DataFrame:")
    print(df.head())
    print("DataFrame shape:", df.shape)
    
    # If df is empty
    if df.empty:
        return pd.DataFrame()
    
    # Ensure the expected columns are present
    expected_cols = {'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'}
    if not expected_cols.issubset(df.columns):
        # No valid data
        return pd.DataFrame()
    
    # Check at least 20 rows for rolling calculation
    if len(df) < 20:
        return pd.DataFrame()

    # Compute 20-day average volume
    df['20d_avg_volume'] = df['Volume'].rolling(window=20).mean()

    # Drop rows without full 20-day average volume
    if '20d_avg_volume' not in df.columns:
        return pd.DataFrame()
    df = df.dropna(subset=['20d_avg_volume'])

    # Check again if we have at least 20 rows
    if df.empty or len(df) < 20:
        return pd.DataFrame()

    # Previous day's close
    df['Prev_Close'] = df['Close'].shift(1)
    df = df.dropna(subset=['Prev_Close'])

    if df.empty:
        return pd.DataFrame()

    # Convert X% to a multiplier
    volume_multiplier = 1 + (X / 100.0)

    # Identify conditions
    df['Volume_Breakout'] = df['Volume'] > (df['20d_avg_volume'] * volume_multiplier)
    df['Price_Change'] = (df['Close'] / df['Prev_Close']) - 1
    df['Price_Breakout'] = df['Price_Change'] >= (Y / 100.0)
    df['Breakout'] = df['Volume_Breakout'] & df['Price_Breakout']

    breakout_days = df[df['Breakout']].index
    results = []

    for d in breakout_days:
        idx = df.index.get_loc(d)
        sell_idx = idx + Z
        if sell_idx < len(df):
            buy_date = d
            sell_date = df.index[sell_idx]
            buy_price = df.loc[buy_date, 'Close']
            sell_price = df.loc[sell_date, 'Close']
            ret = (sell_price - buy_price) / buy_price * 100.0
            results.append({
                'Buy_Date': buy_date,
                'Buy_Price': buy_price,
                'Sell_Date': sell_date,
                'Sell_Price': sell_price,
                'Return_%': ret
            })

    results_df = pd.DataFrame(results)
    results_df.to_csv("breakout_results.csv", index=False)
    return results_df

# Create input widgets
ticker_widget = widgets.Text(value='AAPL', description='Ticker:')
start_date_widget = widgets.DatePicker(value=datetime.date(2020,1,1), description='Start Date:')
end_date_widget = widgets.DatePicker(value=datetime.date(2021,1,1), description='End Date:')
volume_threshold_widget = widgets.FloatText(value=200.0, description='Volume Threshold (%)')
daily_change_threshold_widget = widgets.FloatText(value=2.0, description='Daily Change (%)')
holding_period_widget = widgets.IntText(value=10, description='Holding Period (days)')

run_button = widgets.Button(description='Generate Report', button_style='success')
output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        X = volume_threshold_widget.value
        Y = daily_change_threshold_widget.value
        Z = holding_period_widget.value
        ticker = ticker_widget.value
        start_date_str = start_date_widget.value.strftime("%Y-%m-%d")
        end_date_str = end_date_widget.value.strftime("%Y-%m-%d")

        results_df = backtest_breakout_strategy(X, Y, Z, ticker=ticker, start_date=start_date_str, end_date=end_date_str)

        if not results_df.empty:
            display(results_df)
            avg_return = results_df['Return_%'].mean()
            median_return = results_df['Return_%'].median()
            max_return = results_df['Return_%'].max()
            min_return = results_df['Return_%'].min()
            total_trades = len(results_df)

            print("Summary Statistics:")
            print(f"Total Trades: {total_trades}")
            print(f"Average Return: {avg_return:.2f}%")
            print(f"Median Return: {median_return:.2f}%")
            print(f"Max Return: {max_return:.2f}%")
            print(f"Min Return: {min_return:.2f}%")
            print("CSV saved as breakout_results.csv.")
        else:
            print("No trades found or insufficient data for the given criteria.")

run_button.on_click(on_button_click)

display(widgets.HTML("<h1>Stock Breakout Analyzer</h1><p>Enter parameters and click 'Generate Report'.</p>"))
display(ticker_widget, start_date_widget, end_date_widget, volume_threshold_widget, daily_change_threshold_widget, holding_period_widget, run_button, output)


HTML(value="<h1>Stock Breakout Analyzer</h1><p>Enter parameters and click 'Generate Report'.</p>")

Text(value='AAPL', description='Ticker:')

DatePicker(value=datetime.date(2020, 1, 1), description='Start Date:')

DatePicker(value=datetime.date(2021, 1, 1), description='End Date:')

FloatText(value=200.0, description='Volume Threshold (%)')

FloatText(value=2.0, description='Daily Change (%)')

IntText(value=10, description='Holding Period (days)')

Button(button_style='success', description='Generate Report', style=ButtonStyle())

Output()